# Bach Chorale Generator

In [1]:
import os
import sys
import pathlib
import urllib
import shutil
import re
import zipfile

import numpy as np
import torch
import music21

Create dataset of Chorale phrase tensors. Each item in the dataset is a vector
comprised of several concatenated one-hot vectors, where each one defines a
feature in the phrase

In [2]:
from chorale_phrase_tensor import PhraseTensor
from chorale_feature_extractor import ExtractedChorale, ExtractedPhrase, ExtractedChoralesContainer
import pickle

In [3]:
# Change to False if already have those files

CREATE_EXTRACTED_CHORALE_DATASET = False
CREATE_PHRASE_TENSOR_DATASET = True

In [4]:
if CREATE_EXTRACTED_CHORALE_DATASET:

    # Get an instance of the Chorale iterator, to iterate over all chorales in corpus
    chorales_iterator = music21.corpus.chorales.Iterator()

    # Make a ChoraleVector object for each of the chorales
    extracted_chorale_container = ExtractedChoralesContainer(chorales_iterator)

    # Save all extracted chorales to file for quicker handling
    pickle_file = open('extracted_chorale_container.pkl', 'wb')
    pickle.dump(extracted_chorale_container, pickle_file)
    pickle_file.close()

else:
    pickle_file = open('extracted_chorale_container.pkl', 'rb')
    extracted_chorale_container = pickle.load(pickle_file)
    pickle_file.close()

In [5]:
if CREATE_PHRASE_TENSOR_DATASET:
    # Create a tensor dataset from all phrase tensors extracted from chorales
    phrase_tensors = []
    for chorale_name in extracted_chorale_container.chorale_dict.keys():
        for phrase in extracted_chorale_container.chorale_dict[chorale_name].phrase_vector:
            new_phrase_tensor_object = PhraseTensor(phrase)
            if not new_phrase_tensor_object.valid:
                continue
            phrase_tensors.append(new_phrase_tensor_object().unsqueeze(0))

    phrase_tensor_dataset = torch.utils.data.TensorDataset(torch.concat([*phrase_tensors], dim=0))

    torch.save(phrase_tensor_dataset, 'phrase_tensor_dataset.pt')

else:
    phrase_tensor_dataset = torch.load('phrase_tensor_dataset.pt')

Phrase is longer than maximum allotted. Chorale bwv41.6.mxl phrase index 0
Phrase is longer than maximum allotted. Chorale bwv41.6.mxl phrase index 1
Phrase is longer than maximum allotted. Chorale bwv8.6.mxl phrase index 0
Phrase is longer than maximum allotted. Chorale bwv121.6.mxl phrase index 3
Phrase is longer than maximum allotted. Chorale bwv29.8.mxl phrase index 2
Phrase is longer than maximum allotted. Chorale bwv27.6.mxl phrase index 4
Phrase is longer than maximum allotted. Chorale bwv27.6.mxl phrase index 5
Phrase is longer than maximum allotted. Chorale bwv123.6.mxl phrase index 2
Phrase is longer than maximum allotted. Chorale bwv123.6.mxl phrase index 5
Phrase is longer than maximum allotted. Chorale bwv407.mxl phrase index 9
Phrase is longer than maximum allotted. Chorale bwv320.mxl phrase index 2
Phrase is longer than maximum allotted. Chorale bwv320.mxl phrase index 3
Phrase is longer than maximum allotted. Chorale bwv320.mxl phrase index 4
Phrase is longer than maxim

In [6]:
# Now we train the generator to produce such phrases

# First, set hyperparameters for learning the phrase encodings:
hp = dict(
    batch_size=10, z_dim=15, x_sigma2=1e-4, learn_rate=1e-4, betas=(0.5, 0.999),
)

import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.nn import DataParallel
from trainer import *
import trainer
from phrase_encoder_decoder import VAE, PhraseEncoder, \
    PhraseDecoder, vae_loss, PhraseDecoderWithSoftmax

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Hyperparams
batch_size = hp['batch_size']
z_dim = hp['z_dim']
x_sigma2 = hp['x_sigma2']
learn_rate = hp['learn_rate']
betas = hp['betas']

# Data
train_length = int(len(phrase_tensor_dataset)*0.9)
split_lengths = [train_length, len(phrase_tensor_dataset) - train_length]
ds_train, ds_test = random_split(phrase_tensor_dataset, split_lengths)
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_test  = DataLoader(ds_test,  batch_size, shuffle=True)
im_size = ds_train[0][0].shape[0]

# Model
encoder = PhraseEncoder(im_size, z_dim)
decoder = PhraseDecoderWithSoftmax(z_dim, im_size)
vae = VAE(encoder, decoder, im_size, z_dim)
vae_dp = DataParallel(vae).to(device)

# Optimizer
optimizer = optim.Adam(vae.parameters(), lr=learn_rate, betas=betas)

# Loss
def loss_fn(x, xr, z_mu, z_log_sigma2):
    return vae_loss(x, xr, z_mu, z_log_sigma2, x_sigma2)

# Trainer
trainer = VAETrainer(vae_dp, loss_fn, optimizer, device)
checkpoint_file = 'checkpoints/vae'
checkpoint_file_final = f'{checkpoint_file}_final'
if os.path.isfile(f'{checkpoint_file}.pt'):
    os.remove(f'{checkpoint_file}.pt')

# Show model and hypers
print(vae)
print(hp)

Using device: cpu
VAE(
  (features_encoder): PhraseEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=28, out_features=21, bias=True)
      (1): ReLU()
      (2): Linear(in_features=21, out_features=18, bias=True)
      (3): ReLU()
      (4): Linear(in_features=18, out_features=15, bias=True)
    )
  )
  (features_decoder): PhraseDecoderWithSoftmax(
    (decoder): PhraseDecoder(
      (decoder): Sequential(
        (0): Linear(in_features=15, out_features=18, bias=True)
        (1): ReLU()
        (2): Linear(in_features=18, out_features=21, bias=True)
        (3): ReLU()
        (4): Linear(in_features=21, out_features=28, bias=True)
      )
    )
    (softmax): Softmax(dim=1)
  )
  (encoder_mean): Linear(in_features=15, out_features=15, bias=True)
  (log_sigma2): Linear(in_features=15, out_features=15, bias=True)
)
{'batch_size': 10, 'z_dim': 15, 'x_sigma2': 0.0001, 'learn_rate': 0.0001, 'betas': (0.5, 0.999)}


In [ ]:
def post_epoch_fn(epoch, train_result, test_result, verbose):
    # Plot some samples if this is a verbose epoch
    if verbose:
        samples = vae.sample(n=1)


if os.path.isfile(f'{checkpoint_file_final}.pt'):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} instead of training')
    checkpoint_file = checkpoint_file_final
else:
    res = trainer.fit(dl_train, dl_test,
                      num_epochs=200, early_stopping=20, print_every=10,
                      checkpoints=checkpoint_file,
                      post_epoch_fn=post_epoch_fn)

# Plot images from best model
saved_state = torch.load(f'{checkpoint_file}.pt', map_location=device)
vae_dp.load_state_dict(saved_state['model_state'])


--- EPOCH 1/200 ---


train_batch:   0%|          | 0/186 [00:00<?, ?it/s]

test_batch:   0%|          | 0/21 [00:00<?, ?it/s]


*** Saved checkpoint checkpoints/vae.pt


train_batch:   0%|          | 0/186 [00:00<?, ?it/s]

...

train_batch:   0%|          | 0/186 [00:00<?, ?it/s]

test_batch:   0%|          | 0/21 [00:00<?, ?it/s]

train_batch:   0%|          | 0/186 [00:00<?, ?it/s]

test_batch:   0%|          | 0/21 [00:00<?, ?it/s]

train_batch:   0%|          | 0/186 [00:00<?, ?it/s]

In [ ]:
from ChoralePhraseTensor import GeneratedPhraseBatchTensor

for _ in range(10):
    GeneratedPhraseBatchTensor(vae_dp.module.sample(1)[0]).display_phrase_information_in_a_given_key()